# 패션 상품 Azure Computer Vision 색상 검색 시스템

이 노트북은 Azure Computer Vision으로 추출한 색상 정보를 활용한 패션 상품 검색 및 분석 시스템을 구현합니다.

## 워크플로우 개요
1. **fashion_products.csv** → 기본 상품 데이터
2. **fashion_vision_color_extractor.py** → Azure Computer Vision으로 색상 추출
3. **fashion-sample 인덱스** → product_color 필드 추가 및 색상 정보 업데이트
4. **노트북** → 색상 기반 검색 및 필터링 테스트

## 주요 기능
- Azure Computer Vision 추출 색상 정보 확인
- 색상별 상품 분포 분석
- 색상 기반 상품 검색 및 필터링
- 색상 + 기존 필터 조합 검색

## 환경 설정 및 라이브러리 import

Azure AI Search와 색상 데이터 분석에 필요한 라이브러리들을 import합니다.

In [22]:
import os
import json
import pandas as pd
import numpy as np
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import warnings
warnings.filterwarnings('ignore')

# Azure Search 클라이언트
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery, VectorFilterMode
from azure.search.documents.indexes import SearchIndexClient

# 공통
from dotenv import load_dotenv
load_dotenv(override=True)

# 패션 검색 필터 클래스 정의 (색상 포함)
@dataclass
class FashionColorSearchFilter:
    """패션 색상 검색 필터 클래스"""
    colors: Optional[List[str]] = None  # Azure Computer Vision 추출 색상
    styles: Optional[List[str]] = None
    materials: Optional[List[str]] = None
    seasons: Optional[List[str]] = None
    target_audience: Optional[List[str]] = None
    occasions: Optional[List[str]] = None
    price_ranges: Optional[List[str]] = None
    brands: Optional[List[str]] = None
    categories: Optional[List[str]] = None

# Azure 설정
search_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
index_name = os.getenv("FASHION_INDEX_NAME", "fashion-sample")
admin_key = os.getenv("AZURE_SEARCH_ADMIN_KEY")

# 한국어-영어 색상 매핑 테이블
COLOR_MAPPING = {
    # 기본 색상
    "화이트": "White", "흰색": "White", "흰": "White",
    "블랙": "Black", "검은색": "Black", "검정": "Black",
    "레드": "Red", "빨간색": "Red", "빨강": "Red",
    "블루": "Blue", "파란색": "Blue", "파랑": "Blue",
    "그린": "Green", "초록색": "Green", "초록": "Green",
    "옐로": "Yellow", "노란색": "Yellow", "노랑": "Yellow",
    "핑크": "Pink", "분홍색": "Pink", "분홍": "Pink",
    "퍼플": "Purple", "보라색": "Purple", "보라": "Purple",
    "오렌지": "Orange", "주황색": "Orange", "주황": "Orange",
    "브라운": "Brown", "갈색": "Brown", "갈": "Brown",
    "그레이": "Gray", "회색": "Gray", "회": "Gray",
    "네이비": "Navy", "남색": "Navy",
    "베이지": "Beige", "베이지색": "Beige",
    "골드": "Gold", "금색": "Gold",
    "실버": "Silver", "은색": "Silver"
}

def normalize_color_for_search(color: str) -> str:
    """한국어 색상명을 영어로 변환"""
    return COLOR_MAPPING.get(color, color)

print(f"Search Endpoint: {search_endpoint}")
print(f"Fashion Index Name: {index_name}")
print(f"Color mapping table loaded: {len(COLOR_MAPPING)} mappings")
print(f"Environment loaded successfully!")

Search Endpoint: https://hdfn-test-search.search.windows.net
Fashion Index Name: fashion-sample
Color mapping table loaded: 41 mappings
Environment loaded successfully!


In [23]:
# Azure Search 클라이언트 초기화
search_client = SearchClient(
    endpoint=search_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(admin_key)
)

index_client = SearchIndexClient(
    endpoint=search_endpoint,
    credential=AzureKeyCredential(admin_key)
)

# 인덱스 상태 확인
try:
    index_info = index_client.get_index(index_name)
    print(f"인덱스 '{index_name}' 연결 성공")
    
    # 인덱스 필드 확인 (product_color 필드 포함 여부)
    field_names = [field.name for field in index_info.fields]
    if 'product_color' in field_names:
        print("✅ product_color 필드가 인덱스에 존재합니다")
    else:
        print("❌ product_color 필드가 없습니다. fashion_vision_color_extractor.py를 먼저 실행하세요")
    
    # 인덱스 문서 수 확인
    results = search_client.search(search_text="*", top=1, include_total_count=True)
    total_count = results.get_count()
    print(f"총 문서 수: {total_count:,}개")
    
except Exception as e:
    print(f"인덱스 연결 실패: {e}")
    print("먼저 fashion_index.ipynb를 실행하여 fashion-sample 인덱스를 생성하세요.")

인덱스 'fashion-sample' 연결 성공
✅ product_color 필드가 인덱스에 존재합니다
총 문서 수: 464개


## 색상 정보 분석 및 현황 파악

Azure Computer Vision으로 추출된 색상 정보의 분포를 분석합니다.

In [15]:
def analyze_color_distribution():
    """Azure Computer Vision으로 추출된 색상 분포 분석"""
    
    print("🎨 Azure Computer Vision 색상 추출 결과 분석")
    print("=" * 60)
    
    # product_color 필드가 있는 상품들 조회
    results = search_client.search(
        search_text="*",
        filter="product_color ne null",
        select=["id", "title", "brand", "product_color"],
        top=1000
    )
    
    color_data = []
    products_with_color = 0
    
    for result in results:
        product_color = result.get('product_color', '')
        if product_color:
            products_with_color += 1
            color_data.append({
                'id': result.get('id'),
                'title': result.get('title'),
                'brand': result.get('brand'),
                'product_color': product_color
            })
    
    print(f"색상 정보가 있는 상품: {products_with_color}개")
    
    if not color_data:
        print("❌ 색상 정보를 가진 상품이 없습니다.")
        print("fashion_vision_color_extractor.py를 실행하여 색상을 추출하세요.")
        return None
    
    # 색상 분포 분석 - 단순화된 파싱
    all_colors = []
    
    for item in color_data:
        color_str = item['product_color']
        
        # 새로운 형식: "White Black" 또는 "Red" - 공백으로 구분된 색상명
        if color_str and color_str.strip():
            colors = color_str.split()  # 공백으로 분리
            all_colors.extend(colors)
    
    # 색상 통계 (단순화)
    color_counter = Counter(all_colors)
    
    print(f"\n📊 색상 분포 통계:")
    print(f"   총 추출된 색상: {len(all_colors)}개")
    print(f"   고유 색상 종류: {len(color_counter)}개")
    
    print(f"\n🎯 가장 많은 색상 TOP 10:")
    for color, count in color_counter.most_common(10):
        print(f"   • {color}: {count}개")
    
    return {
        'products_with_color': products_with_color,
        'color_data': color_data,
        'all_colors': color_counter
    }

# 색상 분포 분석 실행
color_analysis = analyze_color_distribution()

🎨 Azure Computer Vision 색상 추출 결과 분석
색상 정보가 있는 상품: 462개

📊 색상 분포 통계:
   총 추출된 색상: 653개
   고유 색상 종류: 10개

🎯 가장 많은 색상 TOP 10:
   • White: 444개
   • Black: 80개
   • Grey: 80개
   • Brown: 20개
   • Pink: 16개
   • Red: 4개
   • Yellow: 3개
   • Blue: 3개
   • Green: 2개
   • Orange: 1개


## 색상 기반 검색 함수 정의

Azure Computer Vision으로 추출된 색상 정보를 활용한 검색 함수를 구현합니다.

In [16]:
def fashion_color_search(
    query: str,
    filters: FashionColorSearchFilter = None,
    search_mode: str = "hybrid",
    top: int = 10,
    include_facets: bool = True
):
    """
    색상 정보를 포함한 패션 상품 검색 함수
    
    Args:
        query: 검색어
        filters: 색상 포함 패션 필터 객체
        search_mode: 검색 모드 (text/vector/hybrid)
        top: 반환할 결과 수
        include_facets: 패싯 정보 포함 여부
    """
    
    # 기본 검색 파라미터 - product_color 필드 포함
    search_params = {
        "search_text": query,
        "top": top,
        "select": [
            "id", "title", "brand", "category", "keyword", 
            "price_range", "normal_price", "main_text",
            # 색상 필드
            "product_color",
            # 기존 패션 필드들
            "style", "color", "material", "targetGender", "targetAge", "season",
            "styleTags", "occasionTags", "ageTags", "genderTags", "seasonTags",
            "sizeRange", "brandPositioning", "rating", "reviewCount"
        ]
    }
    
    # 필터 조건 생성
    filter_conditions = []
    
    if filters:
        # Azure Computer Vision 색상 필터 (product_color 필드 활용)
        # 한국어 색상명을 영어로 변환하여 검색
        if filters.colors:
            color_filters = []
            for color in filters.colors:
                # 한국어 색상명을 영어로 변환
                english_color = normalize_color_for_search(color)
                color_filters.append(f"search.ismatch('{english_color}', 'product_color')")
            filter_conditions.append(f"({' or '.join(color_filters)})")
        
        # 기존 스타일 필터들
        if filters.styles:
            style_filters = []
            for style in filters.styles:
                style_filters.extend([
                    f"search.ismatch('{style}*', 'style')",
                    f"search.ismatch('{style}*', 'styleTags')",
                    f"search.ismatch('{style}*', 'keyword')"
                ])
            filter_conditions.append(f"({' or '.join(style_filters)})")
        
        # 소재 필터
        if filters.materials:
            material_filters = []
            for material in filters.materials:
                material_filters.extend([
                    f"search.ismatch('{material}', 'material')",
                    f"search.ismatch('{material}', 'keyword')"
                ])
            filter_conditions.append(f"({' or '.join(material_filters)})")
        
        # 계절 필터
        if filters.seasons:
            season_filters = []
            for season in filters.seasons:
                season_filters.extend([
                    f"season eq '{season}'",
                    f"search.ismatch('{season}', 'seasonTags')",
                    f"search.ismatch('{season}', 'keyword')"
                ])
            filter_conditions.append(f"({' or '.join(season_filters)})")
        
        # 타겟 고객층 필터
        if filters.target_audience:
            audience_filters = []
            for audience in filters.target_audience:
                audience_filters.extend([
                    f"targetGender eq '{audience}'",
                    f"search.ismatch('{audience}', 'targetAge')",
                    f"search.ismatch('{audience}', 'ageTags')",
                    f"search.ismatch('{audience}', 'genderTags')",
                    f"search.ismatch('{audience}', 'keyword')"
                ])
            filter_conditions.append(f"({' or '.join(audience_filters)})")
        
        # 가격대 필터
        if filters.price_ranges:
            price_filter = " or ".join([f"price_range eq '{pr}'" for pr in filters.price_ranges])
            filter_conditions.append(f"({price_filter})")
        
        # 브랜드 필터
        if filters.brands:
            brand_filter = " or ".join([f"brand eq '{brand}'" for brand in filters.brands])
            filter_conditions.append(f"({brand_filter})")
        
        # 카테고리 필터
        if filters.categories:
            category_filter = " or ".join([f"category eq '{cat}'" for cat in filters.categories])
            filter_conditions.append(f"({category_filter})")
    
    # 필터 조건을 AND로 결합
    if filter_conditions:
        search_params["filter"] = " and ".join(filter_conditions)
    
    # 패싯 추가
    if include_facets:
        search_params["facets"] = [
            "brand,count:20",
            "category,count:20", 
            "price_range,count:20",
            "style,count:15",
            "color,count:15",
            "material,count:15",
            "targetGender,count:10",
            "season,count:8"
        ]
    
    # 검색 모드에 따른 추가 설정
    if search_mode == "hybrid":
        search_params["query_type"] = "semantic"
        search_params["semantic_configuration_name"] = "my-semantic-config"
    
    try:
        results = search_client.search(**search_params)
        return [doc for doc in results]
    except Exception as e:
        print(f"❌ 검색 중 오류 발생: {e}")
        return []

def display_color_search_results(results, show_facets=True, facets=None):
    """색상 정보를 포함한 검색 결과 출력"""
    
    print(f"\n🎨 색상 검색 결과: {len(results)}개 상품 발견\n")
    print("=" * 80)
    
    for i, result in enumerate(results, 1):
        # 기본 정보
        print(f"🏷️  #{i} {result.get('title', 'N/A')}")
        print(f"브랜드: {result.get('brand', 'N/A')}")
        print(f"카테고리: {result.get('category', 'N/A')}")
        print(f"가격대: {result.get('price_range', 'N/A')}")
        
        # Azure Computer Vision 추출 색상 정보 (핵심!)
        if result.get('product_color'):
            print(f"🎨 Azure Vision 색상: {result['product_color']}")
        
        # 기존 패션 필드들
        fashion_info = []
        if result.get('style'):
            fashion_info.append(f"스타일: {result['style']}")
        if result.get('color'):
            fashion_info.append(f"기본색상: {result['color']}")
        if result.get('material'):
            fashion_info.append(f"소재: {result['material']}")
        if result.get('targetGender'):
            fashion_info.append(f"타겟: {result['targetGender']}")
        if result.get('season'):
            fashion_info.append(f"계절: {result['season']}")
        
        if fashion_info:
            print(f"패션정보: {' | '.join(fashion_info)}")
        
        # 상품 설명
        if result.get('main_text'):
            print(f"설명: {result['main_text'][:100]}...")
        
        print("-" * 80)
    
    # 패싯 정보 출력
    if show_facets and facets:
        print("\n필터링 옵션:")
        
        for facet_name in ['brand', 'category', 'price_range', 'style', 'season']:
            if facet_name in facets:
                print(f"\n  {facet_name.upper()}:")
                for item in facets[facet_name][:8]:
                    print(f"  • {item['value']} ({item['count']})")
    
    print("\n" + "=" * 80)

## 색상 기반 검색 테스트

Azure Computer Vision으로 추출된 색상 정보를 활용한 다양한 검색 시나리오를 테스트합니다.

In [18]:
# 색상 기반 검색 시나리오 테스트
print("🎨 Azure Computer Vision 색상 기반 검색 테스트")
print("="*80)

# 1. 특정 색상으로 검색 (Azure Computer Vision 추출 색상 활용)
print("\n1️⃣ 색상별 상품 검색 테스트")
print("-" * 40)

# 주요 색상들로 검색 테스트
test_colors = ["Black", "White", "Blue", "Red", "Brown"]

for color in test_colors:
    print(f"\n🔍 '{color}' 색상 상품 검색:")
    
    color_filter = FashionColorSearchFilter(colors=[color])
    results = fashion_color_search(
        query="*",  # 모든 상품 대상
        filters=color_filter,
        top=3
    )
    
    if results:
        print(f"  ✅ {len(results)}개 상품 발견")
        for result in results[:2]:  # 상위 2개만 표시
            product_color = result.get('product_color', '정보없음')
            print(f"    • {result.get('title', 'N/A')[:50]}... | 색상: {product_color}")
    else:
        print(f"  ❌ {color} 색상 상품을 찾을 수 없습니다")

# 2. 색상 + 검색어 조합
print(f"\n\n2️⃣ 색상 + 검색어 조합 검색")
print("-" * 40)

search_scenarios = [
    ("셔츠", ["White", "Blue"]),
    ("바지", ["Black", "Blue"]),
    ("원피스", ["Red", "Pink"]),
    ("아우터", ["Black", "Brown"])
]

for query, colors in search_scenarios:
    print(f"\n🔍 '{query}' + {colors} 색상 검색:")
    
    color_filter = FashionColorSearchFilter(colors=colors)
    results = fashion_color_search(
        query=query,
        filters=color_filter,
        search_mode="hybrid",
        top=3
    )
    
    if results:
        print(f"  ✅ {len(results)}개 상품 발견")
        for result in results[:2]:
            product_color = result.get('product_color', '정보없음')
            print(f"    • {result.get('title', 'N/A')[:50]}... | 색상: {product_color}")
    else:
        print(f"  ❌ 조건에 맞는 상품을 찾을 수 없습니다")

🎨 Azure Computer Vision 색상 기반 검색 테스트

1️⃣ 색상별 상품 검색 테스트
----------------------------------------

🔍 'Black' 색상 상품 검색:
  ✅ 3개 상품 발견
    • [창주스토어 무역센터점][리이/RE RHEE] HALTER NECK KNITTED TOP ... | 색상: Black Brown
    • [창주스토어 무역센터점][리이/RE RHEE] RERHEE ORGANIC COTTON T ... | 색상: White Black

🔍 'White' 색상 상품 검색:
  ✅ 3개 상품 발견
    • [창주스토어 더현대대구][CC콜렉트] C252PST051 스트라이프 스퀘어넥 탑 ... | 색상: White
    • [창주스토어 무역센터점][Mare di Mari] Stripe Round Neck Top ... | 색상: White

🔍 'Blue' 색상 상품 검색:
  ✅ 3개 상품 발견
    • [창주스토어 천호점]리본 니트넥 티셔츠 RE6-BL260 ... | 색상: White Blue
    • [창주스토어 중동점]리본 니트넥 티셔츠 RE6-BL260 ... | 색상: White Blue

🔍 'Red' 색상 상품 검색:
  ✅ 3개 상품 발견
    • [창주스토어 미아점][타미힐피거여성] 스탬프 로고 폴로셔츠 2색상 6TPI030WT1 ... | 색상: White Red
    • [창주스토어 무역센터점]오일릴리 패턴 티셔츠 OWES2TS041-17 ... | 색상: White Red

🔍 'Brown' 색상 상품 검색:
  ✅ 3개 상품 발견
    • [창주스토어 무역센터점][리이/RE RHEE] HALTER NECK KNITTED TOP ... | 색상: Black Brown
    • [창주스토어 무역센터점][리이/RE RHEE] SQUARE NECK SHIRRED TOP ... | 색상: Brown Black


2️⃣ 색상 + 검색어 조합 검색
-------

In [19]:
# 2-1. 한국어 색상 검색 테스트
print(f"\n\n2️⃣-1 한국어 색상 검색 테스트")
print("-" * 40)

korean_test_colors = ["화이트", "블랙", "핑크", "레드", "블루"]

for korean_color in korean_test_colors:
    english_color = normalize_color_for_search(korean_color)
    print(f"\n🔍 '{korean_color}' ({english_color}) 색상 상품 검색:")
    
    color_filter = FashionColorSearchFilter(colors=[korean_color])
    results = fashion_color_search(
        query="*",
        filters=color_filter,
        top=3
    )
    
    if results:
        print(f"  ✅ {len(results)}개 상품 발견")
        for result in results[:2]:
            product_color = result.get('product_color', '정보없음')
            print(f"    • {result.get('title', 'N/A')[:50]}... | 색상: {product_color}")
    else:
        print(f"  ❌ {korean_color} 색상 상품을 찾을 수 없습니다")



2️⃣-1 한국어 색상 검색 테스트
----------------------------------------

🔍 '화이트' (White) 색상 상품 검색:
  ✅ 3개 상품 발견
    • [창주스토어 더현대대구][CC콜렉트] C252PST051 스트라이프 스퀘어넥 탑 ... | 색상: White
    • [창주스토어 무역센터점][Mare di Mari] Stripe Round Neck Top ... | 색상: White

🔍 '블랙' (Black) 색상 상품 검색:
  ✅ 3개 상품 발견
    • [창주스토어 무역센터점][리이/RE RHEE] HALTER NECK KNITTED TOP ... | 색상: Black Brown
    • [창주스토어 무역센터점][리이/RE RHEE] RERHEE ORGANIC COTTON T ... | 색상: White Black

🔍 '핑크' (Pink) 색상 상품 검색:
  ✅ 3개 상품 발견
    • [창주스토어 중동점]리본 스트라이프 캡소매 니트 티셔츠 RD6-BL300 ... | 색상: White Pink
    • [창주스토어 판교점][베르니스] 엔젤윙 라인스톤 반소매 티셔츠 BE25A3FRR1371 ... | 색상: White Pink

🔍 '레드' (Red) 색상 상품 검색:
  ✅ 3개 상품 발견
    • [창주스토어 미아점][타미힐피거여성] 스탬프 로고 폴로셔츠 2색상 6TPI030WT1 ... | 색상: White Red
    • [창주스토어 무역센터점]오일릴리 패턴 티셔츠 OWES2TS041-17 ... | 색상: White Red

🔍 '블루' (Blue) 색상 상품 검색:
  ✅ 3개 상품 발견
    • [창주스토어 천호점]리본 니트넥 티셔츠 RE6-BL260 ... | 색상: White Blue
    • [창주스토어 중동점]리본 니트넥 티셔츠 RE6-BL260 ... | 색상: White Blue


In [20]:
# 3. 복합 필터링: 색상 + 스타일 + 시즌
print(f"\n3️⃣ 색상 + 패션 속성 복합 필터링")
print("-" * 40)

complex_scenarios = [
    {
        "name": "블랙 캐주얼 여름 아이템",
        "query": "티셔츠",
        "filter": FashionColorSearchFilter(
            colors=["Black"],
            styles=["캐주얼"],
            seasons=["여름"]
        )
    },
    {
        "name": "화이트 오피스 아이템",
        "query": "셔츠",
        "filter": FashionColorSearchFilter(
            colors=["White"],
            target_audience=["여성"]
        )
    },
    {
        "name": "브라운 가을 아우터",
        "query": "코트 자켓",
        "filter": FashionColorSearchFilter(
            colors=["Brown"],
            seasons=["가을"]
        )
    }
]

for scenario in complex_scenarios:
    print(f"\n🔍 {scenario['name']}:")
    
    results = fashion_color_search(
        query=scenario['query'],
        filters=scenario['filter'],
        search_mode="hybrid",
        top=3
    )
    
    if results:
        print(f"  ✅ {len(results)}개 상품 발견")
        display_color_search_results(results[:2], show_facets=False)
    else:
        print(f"  ❌ 조건에 맞는 상품을 찾을 수 없습니다")

# 4. 다중 색상 검색 (OR 조건)
print(f"\n4️⃣ 다중 색상 검색 (여러 색상 중 하나라도 포함)")
print("-" * 40)

multi_color_queries = [
    ("모노톤 아이템", ["Black", "White", "Gray"]),
    ("따뜻한 색감 아이템", ["Red", "Orange", "Brown"]),
    ("차가운 색감 아이템", ["Blue", "Navy", "Green"])
]

for query_name, colors in multi_color_queries:
    print(f"\n🔍 {query_name} ({', '.join(colors)}):")
    
    color_filter = FashionColorSearchFilter(colors=colors)
    results = fashion_color_search(
        query="*",
        filters=color_filter,
        top=5
    )
    
    if results:
        print(f"  ✅ {len(results)}개 상품 발견")
        for result in results[:3]:
            product_color = result.get('product_color', '정보없음')
            print(f"    • {result.get('title', 'N/A')[:50]}... | 색상: {product_color}")
    else:
        print(f"  ❌ 조건에 맞는 상품을 찾을 수 없습니다")


3️⃣ 색상 + 패션 속성 복합 필터링
----------------------------------------

🔍 블랙 캐주얼 여름 아이템:
  ✅ 3개 상품 발견

🎨 색상 검색 결과: 2개 상품 발견

🏷️  #1 [창주스토어 천호점]리본 스트라이프 니트 티셔츠 RE6-BL290 
브랜드: 리본
카테고리: 패션의류
가격대: 7만원대
🎨 Azure Vision 색상: White Black
패션정보: 스타일: 캐주얼 | 기본색상: 미확인 (추가 확인 필요) | 소재: 면혼방 (추정) | 타겟: 여성 | 계절: 여름 및 간절기
설명: [창주스토어 천호점]리본 스트라이프 니트 티셔츠 RE6-BL290  리본 패션의류 심플하면서도 나염 패턴이 돋보이는 민소매 티셔츠로, 여름철 편안한 착용감을 제공하며 실내외 활동에...
--------------------------------------------------------------------------------
🏷️  #2 [창주스토어 신촌점]리본 니트카라 여름 티셔츠 RF6-GB610 
브랜드: 리본
카테고리: 패션의류
가격대: 7만원대
🎨 Azure Vision 색상: Grey White Black
패션정보: 스타일: 캐주얼 | 타겟: 여성 | 계절: 여름
설명: [창주스토어 신촌점]리본 니트카라 여름 티셔츠 RF6-GB610  리본 패션의류 니트 소재의 카라 디자인이 돋보이는 여름용 여성 티셔츠. 여름 니트 카라 티셔츠 캐주얼 캐주얼...
--------------------------------------------------------------------------------


🔍 화이트 오피스 아이템:
  ✅ 3개 상품 발견

🎨 색상 검색 결과: 2개 상품 발견

🏷️  #1 [창주스토어 신촌점]크레송 CRESSON(KDCFBP63)롱 셔츠 
브랜드: 크레송
카테고리: 패션의류
가격대: 10만원대
🎨 Azure Vision 색상: White
패션정보: 스타일: 세미캐주얼 | 기본색상: 뉴트럴 

## Azure Computer Vision 색상 정보 활용도 분석

추출된 색상 정보의 품질과 활용도를 분석합니다.

In [21]:
def analyze_color_quality():
    """Azure Computer Vision 색상 추출 품질 분석"""
    
    print("🔍 Azure Computer Vision 색상 추출 품질 분석")
    print("=" * 60)
    
    # 전체 상품 수 조회
    all_results = search_client.search(
        search_text="*",
        select=["id", "title", "product_color"],
        top=1000
    )
    
    total_products = 0
    products_with_color = 0
    color_quality_stats = {
        'has_dominant_colors': 0,
        'has_accent_color': 0,
        'multiple_colors': 0,
        'single_color': 0
    }
    
    for result in all_results:
        total_products += 1
        product_color = result.get('product_color', '')
        
        if product_color and product_color.strip():
            products_with_color += 1
            
            # 새로운 형식의 색상 정보 분석 (예: "White Black")
            colors = product_color.split()
            if colors:
                color_quality_stats['has_dominant_colors'] += 1
                
                if len(colors) > 1:
                    color_quality_stats['multiple_colors'] += 1
                else:
                    color_quality_stats['single_color'] += 1
    
    # 분석 결과 출력
    coverage_rate = (products_with_color / total_products * 100) if total_products > 0 else 0
    
    print(f"\n📊 색상 추출 커버리지:")
    print(f"   전체 상품: {total_products:,}개")
    print(f"   색상 정보 있음: {products_with_color:,}개")
    print(f"   커버리지: {coverage_rate:.1f}%")
    
    print(f"\n🎯 색상 정보 품질:")
    print(f"   색상 정보 있음: {color_quality_stats['has_dominant_colors']}개")
    print(f"   다중 색상: {color_quality_stats['multiple_colors']}개")
    print(f"   단일 색상: {color_quality_stats['single_color']}개")
    
    # 샘플 색상 정보 표시
    print(f"\n📝 색상 정보 샘플:")
    sample_results = search_client.search(
        search_text="*",
        filter="product_color ne null",
        select=["title", "product_color"],
        top=5
    )
    
    for i, result in enumerate(sample_results, 1):
        title = result.get('title', 'N/A')[:40]
        color = result.get('product_color', 'N/A')
        print(f"   {i}. {title}... | {color}")

# 색상 품질 분석 실행
analyze_color_quality()

🔍 Azure Computer Vision 색상 추출 품질 분석

📊 색상 추출 커버리지:
   전체 상품: 464개
   색상 정보 있음: 462개
   커버리지: 99.6%

🎯 색상 정보 품질:
   색상 정보 있음: 462개
   다중 색상: 176개
   단일 색상: 286개

📝 색상 정보 샘플:
   1. [창주스토어 더현대대구][CC콜렉트] C252PST051 스트라이프 스퀘... | White
   2. [창주스토어 무역센터점][Mare di Mari] Stripe Round... | White
   3. [창주스토어 무역센터점][리이/RE RHEE] BLUR LOGO ORGA... | White
   4. [창주스토어 무역센터점][리이/RE RHEE] HALTER NECK KN... | Black Brown
   5. [창주스토어 무역센터점][리이/RE RHEE] RERHEE ORGANIC... | White Black
